In [ ]:
هدف این این فایل پیش آموزش بخش تمییز دهنده شبکه مولد خصمانه است. تحقیقات قبلی نشان داده است که پیش آموزش کمک می کند که شبکه به پایداری بیشتری در مرحله یادگیری خود دست پیدا کند

In [ ]:
import sys
from chainer import *
from utilities import *
from nmt import Encoder, Decoder, AttentionalNMT
import random
import copy
from rnn_csgan import Discriminator

In [ ]:
در کلاس زیر شبکه مولد خصمانه مسئله ماشین ترجمه عصبی را تعریف می نماییم. شبکه ورودی های لازم را در تابع زیر فراخوانی می کند و دو بخش مولد و بخش تمییز دهنده شبکه نیز تعریف می گردد. در صورتی که قبلا شبکه مولد را با استفاده از مدل ماشین ترجمه عصبی پیش آموزش داده باشیم از آن استفاده می نماییم.

In [ ]:
class CSGANNMT(Chain):
    pass

In [ ]:
def __init__(self, source_vocabulary_size, target_vocabulary_size, embed_size, hidden_size, source_vocabulary, target_vocabulary, source_word2vec, target_word2vec, use_dropout, dropout_rate, generation_limit, use_beamsearch, beam_size, library, pre_nmt):
    super(CSGANNMT, self).__init__(
        generator = AttentionalNMT(source_vocabulary_size, target_vocabulary_size, embed_size, hidden_size, source_vocabulary, target_vocabulary, source_word2vec, target_word2vec, use_dropout, dropout_rate, generation_limit, use_beamsearch, beam_size, library),
        discriminator = Discriminator(embed_size, hidden_size, use_dropout, dropout_rate, library),
    )
    if pre_nmt is not None:
        copy_model(pre_nmt, self.generator)

CSGANNMT.__init__ = __init__

تابع زیر با استفاده از تکه هایی از داده ورودی، خروجی و هدف اقدام به استخراج استیت های ورودی، خروجی و هدف می نماید. در نهایت با فراخوانی بخش تمییز دهنده مقدار خطای شبکه تمییز دهنده و مولد محاسبه می شود. در این گام مقدار خطای شبکه مولد که در هنگام اجرای خود شبکه عصبی آن صورت می گیرد و همچنین خروجی این شبکه خالی برگردانده می شود

In [ ]:
def __call__(self, batch_source, batch_target, batch_output):
    self.reset_states()
    if batch_target is not None:
        source_embed_states = list()
        target_embed_states = list()
        predict_embed_states = list()
        for word in batch_source:
            source_embed_states.append(functions.tanh(self.generator.encoder.word2embed(word)))
        for word in batch_target:
            target_embed_states.append(functions.tanh(self.generator.decoder.word2embed(word)))
        for word in batch_output:
            predict_embed_states.append(functions.tanh(self.generator.decoder.word2embed(word)))
        loss_generator2, loss_discriminator, predicts_discriminator_true, predicts_discriminator_generate = self.discriminator(source_embed_states, target_embed_states, predict_embed_states)
        loss_generator = None
        predicts_generator = None
    else:
        loss_generator = None
        loss_discriminator = None
        predicts_discriminator_true = None
        predicts_discriminator_generate = None
    return loss_generator, loss_discriminator, predicts_generator, predicts_discriminator_true, predicts_discriminator_generate

CSGANNMT.__call__ = __call__

دو تابع زیر یکی استیت های شبکه مولد را ریست می کند و تابع دیگر نیز اقدام به استخراج بردار جاسازی برای کلمات ورودی و هدف می نماید که مقادیر آن بر اساس شبکه مولد محاسبه می گردد. در نهایت نیز مقدار امتیاز تابع تمییز دهنده به عنوان خروجی شبکه بازگردانده می شود

In [ ]:
def reset_states(self):
    self.generator.reset_states()

CSGANNMT.reset_states = reset_states

In [ ]:
def get_score(self, source, target):
    source_embed = list()
    target_embed = list()
    for word in source:
        source_embed.append(functions.tanh(self.generator.encoder.word2embed(word)))
    for word in target:
        target_embed.append(functions.tanh(self.generator.decoder.word2embed(word)))

    _, _, score, _ = self.discriminator(source_embed, target_embed, target_embed)
    return score

CSGANNMT.get_score = get_score

اگر برنامه در گام آموزش باشد، کتابخانه لغات ورودی وخروجی بارگزاری می شود.سپس شبکه ماشین ترجمه عصبی مبتنی بر توجه اجرا شده و مقادیر وزنی از آن تهیه می گردد. بر اساس خروجی این اجرا و ورودی های دیگری همچون لغات ورودی و خروجی و پارامترهای دیگر لازم، شبکه مولد خصمانه که در بالا تعریف گردید را فراخوانی می نماییم. مقدار بهنیه شبکه تمییز دهنده از این طریق محاسبه می گردد. مقدار خطای شبکه تمییز دهنده، شبکه مولد، احتمال پیش بینی شبکه تمییز دهنده و پیش بینی شبکه مولد نیز برای هر بخش از جملات محاسبه می گردد. در نهایت نیز مدل برنامه حاوی لغات ورودی و خروجی، مدل وزن و مقدار بهینه شبکه تمیزز دهنده ذخیره می شود.، 

In [ ]:
def train(config):
    trace("Start Pre-Training ...")
    trace("Loading Vocabulary ...")
    source_vocabulary = Vocabulary.load("{}.source_vocabulary".format(config.pre_model))
    target_vocabulary = Vocabulary.load("{}.target_vocabulary".format(config.pre_model))
    config.source_vocabulary_size = source_vocabulary.size
    config.target_vocabulary_size = target_vocabulary.size

    trace("Making Model ...")
    pre_nmt = AttentionalNMT(config.source_vocabulary_size, config.target_vocabulary_size, config.embed_size, config.hidden_size, source_vocabulary, target_vocabulary, None, None, config.use_dropout, config.dropout_rate, None, False, None, config.library)
    serializers.load_hdf5("{}.weights".format(config.pre_model), pre_nmt)
    
    nmt = CSGANNMT(config.source_vocabulary_size, config.target_vocabulary_size, config.embed_size, config.hidden_size, source_vocabulary, target_vocabulary, None, None, config.use_dropout, config.dropout_rate, None, False, None, config.library, pre_nmt)
    if config.use_gpu:
        cuda.get_device(config.gpu_device).use()
        nmt.to_gpu()

    discriminator_opt = config.discriminator_optimizer
    discriminator_opt.setup(nmt.discriminator)
    discriminator_opt.add_hook(optimizer.WeightDecay(0.0001))

    for epoch in range(config.epoch):
        trace("Epoch {}/{}".format(epoch + 1, config.epoch))
        accum_loss_discriminator = 0.0
        finished = 0
        random.seed(epoch)
        for batch_source, batch_target, batch_output in random_sorted_3parallel_batch(config.source_train, config.target_train, config.output_train, source_vocabulary, target_vocabulary, config.batch_size, config.pooling, config.library):
            loss_generator, loss_discriminator, batch_predict_generator, batch_predict_discriminator_true, batch_predict_discriminator_generate = nmt(batch_source, batch_target, batch_output)
            accum_loss_discriminator += loss_discriminator.data
            nmt.zerograds()
            loss_discriminator.backward()
            discriminator_opt.update()

            for source, target, predict_generator, predict_discriminator_true, predict_discriminator_generate in zip(convert_wordlist(batch_source, source_vocabulary), convert_wordlist(batch_target, target_vocabulary), convert_wordlist(batch_output, target_vocabulary), batch_predict_discriminator_true.data, batch_predict_discriminator_generate.data):
                trace("Epoch {}/{}, Sample {}".format(epoch + 1, config.epoch, finished + 1))
                trace("Source                = {}".format(source))
                trace("Target                = {}".format(target))
                trace("Predict_Generator     = {}".format(predict_generator))
                trace("Predict_Discriminator = True:{} Generate:{}".format(predict_discriminator_true, predict_discriminator_generate))
                finished += 1

        trace("accum_loss_discriminator = {}".format(accum_loss_discriminator))
        trace("Saving Model ...")
        model = "{}.pretrain.{:03d}".format(config.model, epoch + 1)
        source_vocabulary.save("{}.source_vocabulary".format(model))
        target_vocabulary.save("{}.target_vocabulary".format(model))
        serializers.save_hdf5("{}.weights".format(model), nmt)
        serializers.save_hdf5("{}.optimizer_discriminator".format(model), discriminator_opt)

    trace("Finished.")

در گام تست برنامه بعد از بارگزاری لغات ورودی و هدف، شبکه مولد خصمانه را اجرا می نماییم. مقدار وزن بعد از اجرا استخراج می گردد. در نهایت نیز فایل پیش بینی ذخیره می گردد.

In [ ]:
def test(config):
    trace("Loading Vocabulary ...")
    source_vocabulary = Vocabulary.load("{}.source_vocabulary".format(config.model))
    target_vocabulary = Vocabulary.load("{}.target_vocabulary".format(config.model))
    config.source_vocabulary_size = source_vocabulary.size
    config.target_vocabulary_size = target_vocabulary.size

    trace("Loading Model ...")
    nmt = CSGANNMT(config.source_vocabulary_size, config.target_vocabulary_size, config.embed_size, config.hidden_size, source_vocabulary, target_vocabulary, None, None, False, 0.0, config.generation_limit, config.use_beamsearch, config.beam_size, config.library, None)
    if config.use_gpu:
        cuda.get_device(config.gpu_device).use()
        nmt.to_gpu()
    serializers.load_hdf5("{}.weights".format(config.model), nmt)

    trace("Generating Translation ...")
    finished = 0
    
    with open(config.predict_file, 'w') as ft:
        for batch_source in mono_batch(config.source_file, source_vocabulary, 1, config.library):
            trace("Sample {} ...".format(finished + 1))
            _, _, batch_predict, _, _ = nmt(batch_source, None)
            for predict in convert_wordlist(batch_predict, target_vocabulary):
                ft.write("{}\n".format(predict))
                finished += 1

In [ ]:
بر اساس ورودی شبکه تعیین می گردد که برنامه در گام آموزش، اعتبارسنجی و یا تست اجرا می شود. بر اساس هر یک از این حالت یک مدل پیش آموزش و مقادیری از پارامتری ها فراخوانی می شود. سپس یکی از توابع بالا اجرا می گردد.

In [ ]:
if __name__ == "__main__":
    config = Configuration(sys.argv[1], sys.argv[2])
    if config.mode == "train":
        config.pre_model = "{}.{:03d}".format(config.pre_model, config.pre_best_epoch)
        train(config)
    elif config.mode == "test":
        trace("Start Testing ...")
        config.source_file = config.source_test
        config.predict_file = "{}.test_result.beam{}".format(config.model, config.beam_size)
        config.model = "{}.{:03d}".format(config.model, int(sys.argv[3]))
        test(config)
        trace("Finished.")
    elif config.mode == "dev":
        trace("Start Developing ...")
        config.source_file = config.source_dev
        model = config.model
        if len(sys.argv) == 5:
            start = int(sys.argv[3]) - 1
            end = int(sys.argv[4])
        else:
            start = 0
            end = config.epoch
        for i in range(start, end):
            config.model = "{}.{:03d}".format(model, i + 1)
            trace("Model {}/{}".format(i + 1, config.epoch))
            config.predict_file = "{}.dev_result".format(config.model)
            test(config)
        trace("Finished.")

نمونه ای از خروجی شبکه پیش آموزش در حالت یادگیری به شکل زیر است:
    2019-09-19 08:10:47.695670 ... Epoch 1/10
2019-09-19 08:10:48.538258 ... Epoch 1/10, Sample 1
2019-09-19 08:10:48.538314 ... Source                = 誰 が 一番 に 着 く か 私 に は 分か り ま せ ん 。
2019-09-19 08:10:48.538334 ... Target                = i can 't tell who will arrive first .
2019-09-19 08:10:48.538349 ... Predict_Generator     = 
2019-09-19 08:10:48.538378 ... Predict_Discriminator = True:0.48084330558776855 Generate:0.4472368061542511
2019-09-19 08:10:48.538409 ... accum_loss_discriminator = 1.3250393867492676
2019-09-19 08:10:48.538425 ... Saving Model ...
2019-09-19 08:10:53.139828 ... Epoch 2/10
2019-09-19 08:10:53.980821 ... Epoch 2/10, Sample 1
2019-09-19 08:10:53.980874 ... Source                = 誰 が 一番 に 着 く か 私 に は 分か り ま せ ん 。
2019-09-19 08:10:53.980893 ... Target                = i can 't tell who will arrive first .
2019-09-19 08:10:53.980907 ... Predict_Generator     = 
2019-09-19 08:10:53.980933 ... Predict_Discriminator = True:0.9999946355819702 Generate:0.721329391002655
2019-09-19 08:10:53.980961 ... accum_loss_discriminator = 1.277730107307434
2019-09-19 08:10:53.980977 ... Saving Model ...
2019-09-19 08:10:58.551137 ... Epoch 3/10
2019-09-19 08:10:59.384991 ... Epoch 3/10, Sample 1
2019-09-19 08:10:59.385045 ... Source                = 誰 が 一番 に 着 く か 私 に は 分か り ま せ ん 。
2019-09-19 08:10:59.385065 ... Target                = i can 't tell who will arrive first .
2019-09-19 08:10:59.385080 ... Predict_Generator     = 
2019-09-19 08:10:59.385116 ... Predict_Discriminator = True:0.9766238927841187 Generate:4.172325134277344e-07
2019-09-19 08:10:59.385147 ... accum_loss_discriminator = 0.023654043674468994
2019-09-19 08:10:59.385164 ... Saving Model ...
2019-09-19 08:11:03.931765 ... Epoch 4/10
2019-09-19 08:11:04.793428 ... Epoch 4/10, Sample 1
2019-09-19 08:11:04.793483 ... Source                = 誰 が 一番 に 着 く か 私 に は 分か り ま せ ん 。
2019-09-19 08:11:04.793501 ... Target                = i can 't tell who will arrive first .
2019-09-19 08:11:04.793516 ... Predict_Generator     = 
2019-09-19 08:11:04.793543 ... Predict_Discriminator = True:0.9999239444732666 Generate:1.2516975402832031e-06
2019-09-19 08:11:04.793572 ... accum_loss_discriminator = 7.729703793302178e-05
2019-09-19 08:11:04.793588 ... Saving Model ...
2019-09-19 08:11:09.399637 ... Epoch 5/10
2019-09-19 08:11:10.234925 ... Epoch 5/10, Sample 1
2019-09-19 08:11:10.234983 ... Source                = 誰 が 一番 に 着 く か 私 に は 分か り ま せ ん 。
2019-09-19 08:11:10.235003 ... Target                = i can 't tell who will arrive first .
2019-09-19 08:11:10.235018 ... Predict_Generator     = 
2019-09-19 08:11:10.235047 ... Predict_Discriminator = True:0.9999154806137085 Generate:1.2516975402832031e-06
2019-09-19 08:11:10.235079 ... accum_loss_discriminator = 8.57684135553427e-05
2019-09-19 08:11:10.235095 ... Saving Model ...
2019-09-19 08:11:14.812221 ... Epoch 6/10
2019-09-19 08:11:15.656572 ... Epoch 6/10, Sample 1
2019-09-19 08:11:15.656628 ... Source                = 誰 が 一番 に 着 く か 私 に は 分か り ま せ ん 。
2019-09-19 08:11:15.656646 ... Target                = i can 't tell who will arrive first .
2019-09-19 08:11:15.656660 ... Predict_Generator     = 
2019-09-19 08:11:15.656689 ... Predict_Discriminator = True:0.9999075531959534 Generate:1.2516975402832031e-06
2019-09-19 08:11:15.656719 ... accum_loss_discriminator = 9.369265171699226e-05
2019-09-19 08:11:15.656735 ... Saving Model ...
2019-09-19 08:11:20.366878 ... Epoch 7/10
2019-09-19 08:11:21.227653 ... Epoch 7/10, Sample 1
2019-09-19 08:11:21.227709 ... Source                = 誰 が 一番 に 着 く か 私 に は 分か り ま せ ん 。
2019-09-19 08:11:21.227726 ... Target                = i can 't tell who will arrive first .
2019-09-19 08:11:21.227741 ... Predict_Generator     = 
2019-09-19 08:11:21.227768 ... Predict_Discriminator = True:0.9999000430107117 Generate:1.2516975402832031e-06
2019-09-19 08:11:21.227796 ... accum_loss_discriminator = 0.00010122787352884188
2019-09-19 08:11:21.227812 ... Saving Model ...
2019-09-19 08:11:25.844666 ... Epoch 8/10
2019-09-19 08:11:26.701684 ... Epoch 8/10, Sample 1
2019-09-19 08:11:26.701739 ... Source                = 誰 が 一番 に 着 く か 私 に は 分か り ま せ ん 。
2019-09-19 08:11:26.701758 ... Target                = i can 't tell who will arrive first .
2019-09-19 08:11:26.701773 ... Predict_Generator     = 
2019-09-19 08:11:26.701802 ... Predict_Discriminator = True:0.9998928308486938 Generate:1.2516975402832031e-06
2019-09-19 08:11:26.701833 ... accum_loss_discriminator = 0.0001084264658857137
2019-09-19 08:11:26.701850 ... Saving Model ...
2019-09-19 08:11:31.291556 ... Epoch 9/10
2019-09-19 08:11:32.192260 ... Epoch 9/10, Sample 1
2019-09-19 08:11:32.192318 ... Source                = 誰 が 一番 に 着 く か 私 に は 分か り ま せ ん 。
2019-09-19 08:11:32.192337 ... Target                = i can 't tell who will arrive first .
2019-09-19 08:11:32.192351 ... Predict_Generator     = 
2019-09-19 08:11:32.192378 ... Predict_Discriminator = True:0.9998859763145447 Generate:1.2516975402832031e-06
2019-09-19 08:11:32.192407 ... accum_loss_discriminator = 0.00011529973562574014
2019-09-19 08:11:32.192422 ... Saving Model ...
2019-09-19 08:11:36.737228 ... Epoch 10/10
2019-09-19 08:11:37.573403 ... Epoch 10/10, Sample 1
2019-09-19 08:11:37.573470 ... Source                = 誰 が 一番 に 着 く か 私 に は 分か り ま せ ん 。
2019-09-19 08:11:37.573496 ... Target                = i can 't tell who will arrive first .
2019-09-19 08:11:37.573522 ... Predict_Generator     = 
2019-09-19 08:11:37.573553 ... Predict_Discriminator = True:0.9998794794082642 Generate:1.2814998626708984e-06
2019-09-19 08:11:37.573598 ... accum_loss_discriminator = 0.00012184485967736691
2019-09-19 08:11:37.573619 ... Saving Model ...
2019-09-19 08:11:42.184670 ... Finished.